# MedMask Library Advantages Demo

This example demonstrates the advantages of MedMask library for medical image segmentation mask processing.

## Core Benefits

1. **File Management** - 117 scattered files → 1 archive file
2. **Storage Optimization** - Zstandard compression, significant space reduction
3. **Semantic Mapping** - Built-in bidirectional name-to-label mapping
4. **Performance Boost** - Optimized I/O and batch operations
5. **Simple API** - Complex operations in one line of code
6. **Overlapping Masks** - Multi-granularity mask combinations

## Data

Using TotalSegmentator's 117 organ segmentation results to compare two processing approaches.


In [2]:
import os
import time
import numpy as np
import nibabel as nib
from pathlib import Path

# Import MedMask library
import sys
sys.path.append('../')
from medmask import SegmentationMask, MaskArchive
from spacetransformer import Space

# Data preparation
data_dir = Path("s0000")
all_files = sorted(data_dir.glob("*.nii.gz"))
first_file = nib.load(all_files[0])
space = Space.from_nifti(first_file)

print(f"Data: {len(all_files)} organ files")
print(f"Image: {first_file.get_fdata().shape} @ {space.spacing[0]:.1f}mm")

# Calculate traditional approach total size
traditional_size = sum(f.stat().st_size for f in all_files)
print(f"Traditional total size: {traditional_size / (1024*1024):.2f} MB")


Data: 117 organ files
Image: (294, 192, 179) @ 1.5mm
Traditional total size: 5.12 MB


## Two Processing Approaches Comparison

### Approach 1: Independent Semantic Masks
Each organ as independent `SegmentationMask`, easier individual access and modification, larger size, suitable for overlapping mask management

### Approach 2: Merged Multi-label Mask  
All organs merged into one multi-label mask, using `add_label()` for unified management, smaller size, suitable for non-overlapping mask management


In [3]:
# Approach 1: Independent semantic masks
print("Approach 1: Creating independent mask archive...")

read_time = time.time()
all_mask_data = {}
for i, organ_file in enumerate(all_files):
    nii = nib.load(organ_file)
    mask_data = np.array(nii.dataobj)
    all_mask_data[organ_file] = mask_data
read_time = time.time() - read_time

start_time = time.time()
archive1 = MaskArchive("organs_individual.mska", mode="w", space=space)
for i, organ_file in enumerate(all_files):
    mask_data = all_mask_data[organ_file]
    organ_name = organ_file.stem.replace('.nii', '')
    
    mask = SegmentationMask(
        mask_array=mask_data.astype(np.uint8),
        mapping={organ_name: 1},
        space=space
        )
    archive1.add_segmask(mask, organ_name)

time1 = time.time() - start_time
size1 = Path("organs_individual.mska").stat().st_size

# Approach 2: Merged multi-label mask
print("Approach 2: Creating merged mask archive...")
start_time = time.time()

combined_mask = SegmentationMask.lazy_init(bit_depth=8, space=space)
all_keys = []
for i, organ_file in enumerate(all_files):
    nii = nib.load(organ_file)
    mask_data = nii.get_fdata() > 0
    organ_name = organ_file.stem.replace('.nii', '')
    all_keys.append(organ_name)
    combined_mask.add_label(mask_data, i + 1, organ_name)

combined_mask.save("organs_combined.msk")

time2 = time.time() - start_time
size2 = Path("organs_combined.msk").stat().st_size

print(f"\nCreation time: Approach 1 {time1:.1f}s, Approach 2 {time2:.1f}s")
print(f"File size: Approach 1 {size1/1024:.0f}KB, Approach 2 {size2/1024:.0f}KB")
print(f"Compression ratio: Approach 1 {traditional_size/size1:.1f}:1, Approach 2 {traditional_size/size2:.1f}:1")


Approach 1: Creating independent mask archive...


/tmp/ipykernel_3141567/4062836606.py:8: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  mask_data = np.array(nii.dataobj)


Approach 2: Creating merged mask archive...

Creation time: Approach 1 8.1s, Approach 2 6.4s
File size: Approach 1 158KB, Approach 2 91KB
Compression ratio: Approach 1 33.1:1, Approach 2 57.6:1


In [4]:
# Reading performance test
test_organs = all_keys

# Test traditional approach
traditional_read_time = read_time

# Test approach 1 - independent masks
start = time.time()
archive1_read = MaskArchive("organs_individual.mska", mode="r")
for organ in test_organs:
    mask = archive1_read.load_segmask(organ)
    data = mask.data
method1_read_time = time.time() - start

# Test approach 2 - merged mask
start = time.time()
all_organs = SegmentationMask.load("organs_combined.msk")
for organ in test_organs:
    data = all_organs.get_binary_mask_by_names(organ)
method2_read_time = time.time() - start

print(f"Reading {len(test_organs)} organs:")
print(f"Traditional approach: {traditional_read_time:.3f}s")
speedup1 = traditional_read_time / method1_read_time
print(f"Approach 1 (independent masks): {method1_read_time:.3f}s ({speedup1:.1f}x speedup)")
speedup2 = traditional_read_time / method2_read_time
print(f"Approach 2 (merged mask): {method2_read_time:.3f}s ({speedup2:.1f}x speedup)")



Reading 117 organs:
Traditional approach: 1.870s
Approach 1 (independent masks): 0.131s (14.2x speedup)
Approach 2 (merged mask): 0.126s (14.8x speedup)


## Overlapping Masks Advantage - Rib Example

Demonstrates storing multiple granularity masks in one archive:
- 24 individual ribs
- Left/right rib combinations  
- Upper/lower rib combinations
- All ribs merged

Supports multi-level queries from individual ribs to global rib mask.


In [5]:
# Rib mask combination example
rib_files = sorted(data_dir.glob("rib_*.nii.gz"))
ribs_archive = MaskArchive("ribs_archive.mska", mode="w", space=space)

# Prepare combination masks
shape = first_file.get_fdata().shape
left_ribs = np.zeros(shape, dtype=bool)
right_ribs = np.zeros(shape, dtype=bool)
all_ribs = np.zeros(shape, dtype=bool)
upper_ribs = np.zeros(shape, dtype=bool)  
lower_ribs = np.zeros(shape, dtype=bool)

print(f"Processing {len(rib_files)} rib files...")

# Process each rib
for rib_file in rib_files:
    nii = nib.load(rib_file)
    rib_data = nii.get_fdata() > 0
    rib_name = rib_file.stem.replace('.nii', '')
    
    # Add individual rib
    mask = SegmentationMask(rib_data.astype(np.uint8), {rib_name: 1}, space)
    ribs_archive.add_segmask(mask, rib_name)
    
    # Combine to group masks
    all_ribs |= rib_data
    if 'left' in rib_name:
        left_ribs |= rib_data
    elif 'right' in rib_name:
        right_ribs |= rib_data
    
    # Upper/lower grouping
    for i in range(1, 13):
        if f'_{i}' in rib_name:
            if i <= 6:
                upper_ribs |= rib_data
            else:
                lower_ribs |= rib_data
            break

# Add combination masks
combinations = [
    ("all_ribs", all_ribs), ("left_ribs", left_ribs), ("right_ribs", right_ribs),
    ("upper_ribs", upper_ribs), ("lower_ribs", lower_ribs)
]

for name, mask_data in combinations:
    if np.any(mask_data):
        mask = SegmentationMask(mask_data.astype(np.uint8), {name: 1}, space)
        ribs_archive.add_segmask(mask, name)

rib_size = Path("ribs_archive.mska").stat().st_size
print(f"Rib archive: {len(ribs_archive.all_names())} masks, {rib_size/1024:.0f}KB")


Processing 24 rib files...
Rib archive: 27 masks, 14KB


## Use Cases

**Independent Masks** suited for:
- Frequent access to individual organs
- Independent modification of specific regions
- Modular data management

**Merged Masks** suited for:
- Global organ analysis
- Unified label management
- Maximum storage efficiency

**Overlapping Masks** support:
- Multi-granularity anatomical queries
- Hierarchical organ organization
- Flexible region combination analysis

---

MedMask provides comprehensive solutions for medical image segmentation masks through advanced compression, semantic design, and flexible APIs.
